# Support Vector Machines

## A Brief History of the Support Vector Machine (SVM)

The Support Vector Machine (SVM) algorithm has a different lineage than statistically-rooted methods like logistic regression. Its development is a story of deep theoretical work on learning theory and optimization, culminating in a practical and powerful algorithm that dominated machine learning for many years.

Here is a timeline of its key developments:

*   **1963: The Linear Classifier**
    *   The foundational ideas were first introduced by **Vladimir Vapnik** and **Alexey Chervonenkis**.
    *   They proposed the "optimal hyperplane" algorithm, which is the basis for the linear SVM. The core idea was to find a decision boundary (a line, or hyperplane in higher dimensions) that is maximally far from the data points of either class. This distance is called the **margin**, and the goal was to maximize it. This was initially a linear classifier for perfectly separable data.

*   **1970s: The Theoretical Underpinnings (VC Theory)**
    *   Throughout the 1970s, Vapnik and Chervonenkis developed what is now known as **Vapnik-Chervonenkis (VC) Theory**, or more broadly, Statistical Learning Theory.
    *   This theory provided a rigorous mathematical framework for understanding how a model generalizes from training data to unseen data. It introduced the concept of the **VC Dimension**, a measure of a model's capacity or complexity. This work established the principle of **Structural Risk Minimization (SRM)**, which aims to balance model complexity with its error on the training data, providing strong theoretical justification for the maximum-margin approach.

*   **1992: The Kernel Trick for Non-Linearity**
    *   The algorithm's true power was unlocked in a groundbreaking paper by **Bernhard Boser, Isabelle Guyon, and Vladimir Vapnik**.
    *   They introduced the **"kernel trick."** The problem was that the optimal hyperplane was a linear separator, but most real-world data isn't linearly separable.
    *   The kernel trick is a mathematically elegant solution: it allows the algorithm to operate in a very high-dimensional feature space *without ever explicitly calculating the coordinates of the data in that space*. It does this by using a kernel function (like a Polynomial or Radial Basis Function) to calculate the dot products between images of data points in the feature space, which is all the algorithm needs. This made it possible to find complex, non-linear decision boundaries efficiently.

*   **1995: The Soft Margin Classifier**
    *   Another major practical limitation was that the original algorithm required the data to be perfectly separable (even in the higher-dimensional space).
    *   **Corinna Cortes** and **Vapnik** introduced the "soft margin" formulation. This modified the optimization problem to allow for some data points to be misclassified or fall within the margin.
    *   It introduced a "cost" hyperparameter, often denoted as `C`, that controls the trade-off between maximizing the margin and minimizing the classification error. A small `C` allows for a wider margin at the cost of more margin violations, while a large `C` penalizes violations heavily, leading to a narrower margin. This made SVMs robust to noisy data and applicable to virtually any dataset.

*   **Late 1990s - 2000s: The Golden Age**
    *   With the combination of the kernel trick and soft margins, SVMs became the state-of-the-art for many classification tasks.
    *   They achieved top performance in areas like handwriting recognition (outperforming early neural networks), text categorization, and bioinformatics. Their strong theoretical backing and excellent empirical performance made them a dominant force in the machine learning community.

*   **Present Day: A Powerful and Relevant Tool**
    *   While Deep Learning and large neural networks have since become the state-of-the-art for unstructured data problems (like image, audio, and natural language processing), SVMs remain a powerful and relevant tool.
    *   They are particularly effective for classification tasks with high-dimensional, structured data, and often perform very well on small-to-medium-sized datasets where deep learning models might overfit. They are still a go-to algorithm and an important benchmark in any machine learning practitioner's toolkit.

## Motivation: Why Support Vector Machines?

Support Vector Machines (SVMs) are a powerful class of supervised learning algorithms used for classification and regression tasks. While models like logistic regression are probabilistic, SVMs are deterministic and based on geometrical properties of the data. The core idea behind SVMs is to find an optimal hyperplane that best separates different classes of data points. This 'optimality' is achieved by maximizing the margin, which is the distance between the hyperplane and the closest data points from each class. These closest points are called **support vectors**, and they are the critical elements of the training data that define the decision boundary.

- Maximizes **margin** → better generalization.
- Works in high-dimensional spaces (e.g., gene expression, spectroscopy)
- Flexible with kernels for non-linear separation.
- Robust to outliers with soft margins.
- The number of features > number of samples (common in biochemistry, genomics).

<div style="text-align: center;">
<figure>
<img src="https://upload.wikimedia.org/wikipedia/commons/7/72/SVM_margin.png" width=60%>
<figcaption> https://upload.wikimedia.org/wikipedia/commons/7/72/SVM_margin.png </figcaption>
</figure>
</div>



Let's see a comparison between SVM and logistic regression for two different data samples:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_circles
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Generate linear data
X_lin, y_lin = make_classification(n_samples=100, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, class_sep=2.0, random_state=42)

# Generate non-linear data
X_nonlin, y_nonlin = make_circles(n_samples=100, factor=0.5, noise=0.1, random_state=42)

def plot_decision_boundary(clf, X, y, ax, title):
    h = .02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3)
    ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k')
    ax.set_title(title)

# Fit models
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for data, ax_row, kernel in zip([(X_lin, y_lin), (X_nonlin, y_nonlin)], axes, ['linear', 'rbf']):
    X, y = data
    log_reg = LogisticRegression().fit(X, y)
    svm = SVC(kernel=kernel).fit(X, y)
    plot_decision_boundary(log_reg, X, y, ax_row[0], 'Logistic Regression')
    plot_decision_boundary(svm, X, y, ax_row[1], f'SVM ({kernel} kernel)')
plt.tight_layout()
plt.show()

## SVM Fundamentals
SVM finds the hyperplane that maximizes the margin (distance to nearest data points of each class). These nearest points are called support vectors.

Given a dataset $(x_i, y_i)$ with $y_i \in \{-1, 1\}$, SVM solves:
$$
\min_{w,b} \frac{1}{2} ||w||^2 \quad \text{s.t.} \quad y_i (w \cdot x_i + b) \geq 1,
$$
- $ \mathbf{w} $: normal vector to hyperplane
- $ b $: bias
- $ y_i \in \{-1, +1\} $
- Margin = $ \frac{2}{\|\mathbf{w}\|} $

This is the so-called hard-margin classifier. The actual hyperplane might not always exists, like in the case of the "circular boundary". 

**Soft-margin** 

When there is not linear (hyperplane boundary), a hinge-loss function is introduced,
\begin{equation}
\max(0, 1 - y_i(w\cdot x_i - b)),
\end{equation}
This function is 0 if the constrain is sattisfied, otherwise its value is proportional to the distance to the margin. This also allows for some misclassification, but this also makes the method less sentitive to outliers. The goal now is to minimize 
\begin{equation}
\|\mathbf{w}\|^{2} 
+ C \left[ \frac{1}{n} \sum_{i=1}^{n} 
\max \bigl(0,\, 1 - y_i(\mathbf{w}^\top \mathbf{x}_i - b)\bigr) \right],
\end{equation}
where $C \lt 0$ represents the trade-off between increasing the margin size and ansuring that $x_i$ ies on the correct size of the boundary.

In this case, soft-margin SVM introduces slack variables $\zeta_i$ and penalty $C$, and the problem becomes

\begin{equation}
\begin{aligned}
\min_{\mathbf{w},\, b,\, \boldsymbol{\zeta}} \quad 
& \|\mathbf{w}\|_{2}^{2} + C \sum_{i=1}^{n} \zeta_i \\[6pt]
\text{subject to} \quad 
& y_i \bigl(\mathbf{w}^\top \mathbf{x}_i - b\bigr) \geq 1 - \zeta_i, 
\quad \zeta_i \geq 0, 
\quad \forall i \in \{1, \ldots, n\}.
\end{aligned}

\end{equation}



:::{exercise}
Derive the dual form of the SVM optimization problem and explain the role of Lagrange multipliers.
:::

### Visualizing the Margin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs

# Create linearly separable data
X, y = make_blobs(n_samples=100, centers=2, random_state=6)

# Fit the SVM model
clf = svm.SVC(kernel='linear', C=100)
clf.fit(X, y)

# Plot the data points
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# Plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# Plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# Plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.title('Linearly Separable Data with SVM')
plt.show()

:::{exercise} Exploring the Margin
1.  Modify the `C` parameter in the `svm.SVC` function in the code above. What do you observe about the margin and the number of support vectors when you use a very small `C` (e.g., 0.01) versus a very large `C` (e.g., 10000)?
2.  What does the `C` parameter control in the context of the SVM classifier? *Hint: Think about the trade-off between a smooth decision boundary and classifying training points correctly.*
:::

:::{exercise}Identify Support Vectors
Use `svm.support_vectors_` to extract and plot the support vectors on the above figure.
:::

## Comparison with Other Methods

### SVM vs. Logistic Regression

| Feature | Support Vector Machine (SVM) | Logistic Regression |
|---|---|---|
| **Underlying Principle** | Geometric: finds the optimal hyperplane that maximizes the margin between classes. | Statistical: models the probability of a certain class or event existing. |
| **Decision Boundary** | Can be linear or non-linear using the kernel trick. | Fundamentally linear, though can be extended to non-linear with feature engineering. |
| **Sensitivity to Outliers**| Less sensitive to outliers due to the margin-based optimization.  | Can be more sensitive to outliers as it tries to classify all points correctly. |
| **Use Cases** | Effective in high-dimensional spaces and for both linear and non-linear problems. Works well with unstructured data like text and images. | Performs well on linearly separable data and when a probabilistic interpretation is needed. Often a good first model to try.  |
| **Robust to overfitting** | Strong (especially in high-dim) | Moderate |

In general, if the number of features is much larger than the number of training examples, logistic regression or a linear SVM is recommended. If the number of examples is intermediate and the data is not linearly separable, an SVM with a non-linear kernel is a good choice.

## The Kernel Trick: Handling Non-Linear Data

What if the data is not linearly separable? This is where the **kernel trick** comes in. The kernel trick is a powerful technique that allows SVMs to classify non-linear data.  It works by mapping the input data into a higher-dimensional space where a linear separator can be found. This is done implicitly, without ever having to compute the coordinates of the data in this higher-dimensional space, which is computationally efficient.

Commonly used kernels include:
*   **Linear:** For linearly separable data.
*   **Polynomial:** Creates a polynomial decision boundary. $ K(x_i, x_j) = (\gamma x_i^T x_j + r)^d $
*   **Radial Basis Function (RBF):** A popular choice for its flexibility in handling complex relationships. $ K(x_i, x_j) = \exp(-\gamma \|x_i - x_j\|^2) $
*   **Sigmoid:** Can be useful in certain neural network-like scenarios.

```{note}
**RBF is default**: Handles complex, non-linear patterns (e.g., in metabolomics, protein folding)
```


In [ ]:
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=100, factor=.1, noise=.1, random_state=42)

# Fit the SVM model with an RBF kernel
clf = svm.SVC(kernel='rbf', C=10, gamma='auto', verbose=False)
clf.fit(X, y)

# Plot the data points
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# Plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# Plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# Plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.title('Non-Linearly Separable Data with RBF Kernel SVM')
plt.show()

:::{exercise}Experimenting with Kernels

1.  In the code above, change the `kernel` parameter to `'linear'` and `'poly'`. How does the decision boundary change? Which kernel performs best for this dataset?
2.  For the RBF kernel, experiment with different values for the `gamma` parameter. What is the effect of a very small `gamma` versus a very large `gamma` on the decision boundary? What might this imply about the model's complexity and potential for overfitting?
:::

## A more complex boundary

In [ ]:
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


# Generate non-linear data
scaler = StandardScaler()
X_moon, y_moon = make_moons(n_samples=100, noise=0.2, random_state=42)
X_moon_scaled = scaler.fit_transform(X_moon)

# Fit SVM with RBF kernel
svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_rbf.fit(X_moon_scaled, y_moon)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(X_moon_scaled[y_moon == 0, 0], X_moon_scaled[y_moon == 0, 1], c='red', label='Class 0', alpha=0.7)
plt.scatter(X_moon_scaled[y_moon == 1, 0], X_moon_scaled[y_moon == 1, 1], c='blue', label='Class 1', alpha=0.7)

# Decision boundary (grid)
xx, yy = np.meshgrid(np.linspace(X_moon_scaled[:, 0].min(), X_moon_scaled[:, 0].max(), 100),
                     np.linspace(X_moon_scaled[:, 1].min(), X_moon_scaled[:, 1].max(), 100))
Z = svm_rbf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contour(xx, yy, Z, levels=[0], colors='k', linestyles='-', alpha=0.7)
plt.contourf(xx, yy, Z, levels=50, alpha=0.3, cmap='RdBu')

plt.xlabel('Feature 1 (scaled)')
plt.ylabel('Feature 2 (scaled)')
plt.title('SVM with RBF Kernel: Non-Linear Separation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


:::{exercise} Compare SVM vs Logistic Regression
Fit a logistic regression model on the same moons dataset. Compare accuracy and decision boundary.
:::

In [ ]:
from sklearn.linear_model import LogisticRegression
# YOUR CODE HERE


print(f"Logistic Regression Accuracy: {accuracy_score(y_moon, y_pred_lr):.3f}")
print(f"SVM (RBF) Accuracy: {accuracy_score(y_moon, svm_rbf.predict(X_moon_scaled)):.3f}")



### Interactive role of hyper parameters

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
import ipywidgets as widgets
from IPython.display import display


In [ ]:
# Generate synthetic 2D data
X, y = make_classification(n_samples=150, n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, class_sep=1.5, random_state=42)

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create interactive controls
@widgets.interact(
    C=widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='C'),
    gamma=widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Gamma'),
    kernel=widgets.Dropdown(options=['rbf', 'linear', 'poly'], value='rbf', description='Kernel')
)
def plot_svm(C=1.0, gamma=0.1, kernel='rbf'):
    # Fit SVM
    svm = SVC(kernel=kernel, C=C, gamma=gamma, random_state=42)
    svm.fit(X_scaled, y)
    
    # Plot
    plt.figure(figsize=(10, 7))
    plt.scatter(X_scaled[y == 0, 0], X_scaled[y == 0, 1], c='red', label='Class 0', alpha=0.7)
    plt.scatter(X_scaled[y == 1, 0], X_scaled[y == 1, 1], c='blue', label='Class 1', alpha=0.7)
    
    # Decision boundary (grid)
    xx, yy = np.meshgrid(np.linspace(X_scaled[:, 0].min(), X_scaled[:, 0].max(), 100),
                         np.linspace(X_scaled[:, 1].min(), X_scaled[:, 1].max(), 100))
    Z = svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contour(xx, yy, Z, levels=[0], colors='k', linestyles='-', alpha=0.8)
    plt.contourf(xx, yy, Z, levels=50, alpha=0.3, cmap='RdBu')
    
    # Support vectors
    sv = svm.support_vectors_
    plt.scatter(sv[:, 0], sv[:, 1], s=100, facecolors='none', edgecolors='green', linewidth=2, label='Support Vectors')
    
    plt.xlabel('Feature 1 (scaled)')
    plt.ylabel('Feature 2 (scaled)')
    plt.title(f'SVM Decision Boundary (C={C}, gamma={gamma:.2f}, kernel={kernel})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## Applications in Basic Sciences

SVMs have found numerous applications in various scientific domains:

*   **Bioinformatics:** SVMs are widely used for tasks like protein classification, gene expression analysis, and cancer classification. Their ability to handle high-dimensional data makes them suitable for analyzing genomic and proteomic datasets.
*   **Image Classification:** In fields like medical imaging and satellite imagery analysis, SVMs can be used to classify images, for instance, to identify tumors in medical scans or to classify different types of land cover from satellite data. 
*   **Chemistry:** SVMs can be used in cheminformatics to predict the properties of molecules, such as their bioactivity or toxicity, based on their chemical structure.

### Biochemistry: Enzyme Substrate Classification
Features: 3D molecular descriptors (e.g., hydrophobicity, size, charge).
Task: Classify if a molecule is a substrate (yes/no) for an enzyme.
Why SVM? High-dim, small samples, non-linear relationships

### Spectroscopy: Material Phase Classification
Features: Raman or IR intensity at 1000+ wavenumbers.
Task: Classify solid vs. liquid phase.
Why SVM? RBF kernel captures subtle spectral shifts.

:::{exercise}
Dataset: breast_cancer from sklearn (classic biomedical dataset).
Goal: Classify tumors as malignant/benign using 30 morphological features.
:::

In [ ]:
# YOUR CODE HERE



## EXTRA: scikit pipeline
 A full scikit pipeline allows you to estimate the best hyper parameters for a given problem. It uses <https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html>

In [ ]:
from sklearn.metrics import accuracy_score
# Build full pipeline
def create_svm_pipeline():
    return Pipeline([
        ('scaler', StandardScaler()),
        ('feature_selection', SelectKBest(f_classif, k=10)),  # Select top 10 features
        ('svm', SVC(kernel='rbf', random_state=42))
    ])

# Use on breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create pipeline
pipe = create_svm_pipeline()

# Hyperparameter tuning
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'svm__kernel': ['rbf', 'poly'],
    'feature_selection__k': [5, 10, 15]
}

# Grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best CV score:", grid.best_score_.round(3))

# Final prediction
y_pred = grid.predict(X_test)
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malignant']))


Comparing svm and logistic regression

In [ ]:
# Logistic Regression pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def compare_models(X_train, X_test, y_train, y_test):
    models = {
        'SVM (RBF)': Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif, k=10)),
            ('clf', SVC(kernel='rbf', C=10, gamma=0.01, probability=True))
        ]),
        'Logistic Regression': Pipeline([
            ('scaler', StandardScaler()),
            ('selector', SelectKBest(f_classif, k=10)),
            ('clf', LogisticRegression(max_iter=1000))
        ])
    }
    
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        results[name] = {'test_score': score, 'cv_mean': cv_scores.mean(), 'cv_std': cv_scores.std()}
        print(f"{name}: Test Accuracy = {score:.3f}, CV Accuracy = {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    
    return results

# Compare on breast cancer
compare_models(X_train, X_test, y_train, y_test)


:::{exercise} Build Your Own Pipeline
Use make_classification to generate a dataset with 100 features, 500 samples, and non-linear structure.
:::

In [ ]:
# YOUR CODE HERE



## EXTRA: Support Vector Machines (SVM) with Model Explainability (SHAP)
Please look at <https://shap.readthedocs.io/en/latest/index.html>
